In [1]:
import pandas as pd
import numpy as np
import altair as alt

In [2]:
hexagon = "M0,-2.3094010768L2,-1.1547005384 2,1.1547005384 0,2.3094010768 -2,1.1547005384 -2,-1.1547005384Z"

In [3]:
converter = "/Users/xavier/Desktop/DSPP/solo_projects/redistricting_project/app_building/app_data/converter.csv"
state_data = "/Users/xavier/Desktop/DSPP/solo_projects/redistricting_project/app_building/app_data/state_data.csv"
state_coords = "/Users/xavier/Desktop/DSPP/solo_projects/redistricting_project/app_building/app_data/cartogram_coords.csv"
hourglass_coords = "/Users/xavier/Desktop/DSPP/solo_projects/redistricting_project/app_building/app_data/hourglass_coords.csv"
district_data = "/Users/xavier/Desktop/DSPP/solo_projects/redistricting_project/app_building/app_data/district_data.csv"

In [4]:
cv = pd.read_csv(converter)
st = pd.read_csv(state_data)
sc = pd.read_csv(state_coords)
hc = pd.read_csv(hourglass_coords)
dd = pd.read_csv(district_data)

In [5]:
#create a function to change the probabilities based on a partisan wave

def call_wave(wave):
    '''Call a model by function name (to be used within a larger function)
    
    Args:
        A string that refers to the model name. Options are Linear SVC, SVC, and Logistic Regression 
    
    Returns:
        A model as a function for later use
    '''
    #create an if/else to identify the possible models
    if wave == "Republican":
        col = cv["logit_red"]
    elif wave == "Democratic":
        col = cv["logit_blue"]
    #return a string if it fails- it will block the function from applying
    else:
        col = cv["logit_all"]
    index = cv[["metric"]]
    index["prob"] = col
    return index

In [6]:
st = st.merge(sc, how="left")

## Create the Function

In [7]:
def get_viz(year,wave,chart):
    #build out the chart
    df = st[st["year"] == year]
    df["ST"] = df["ST"]
    #state cartogram
    #build the chart
    st_cart = alt.Chart(df).mark_point().encode(
        alt.X('X',axis=None),
        alt.Y('Y',axis=None),
        alt.Text('ST'),
        alt.Color('porp_loss', scale=alt.Scale(scheme='redblue'), legend=alt.Legend(title="Porportion not Represented")),
        tooltip = [alt.Tooltip('porp_text',title='Porportion Lost'),
                   alt.Tooltip('dist_text',title='District Deficit')
                  ]
    )
    #inser labels
    st_cart_labs = alt.Chart(df).mark_text(align='center').encode(
        alt.X('X'),
        alt.Y('Y'),
        alt.Text('ST'),
    ).properties(title='Severity of Partisan Misrepresentation')
    #display
    cart = alt.layer(st_cart, st_cart_labs).configure_axis(grid=False).configure_view(strokeWidth=0).configure_point(size=250,shape=hexagon,filled=True)
    #create a bubble chart
    selection = alt.selection_single();
    bubble = alt.Chart(df).mark_point(filled=True).encode(
        alt.X('metric', scale=alt.Scale(zero=False), title='Porportion Republican'),
        alt.Y('porp_loss', scale=alt.Scale(zero=False), title='Porportion of Population not Represented'),
        alt.Size('dist_loss', legend=alt.Legend(title="Amount of Seats Effected")),
        alt.Order('dist_loss', sort='descending'),
        alt.Color('porp_loss', scale=alt.Scale(scheme='redblue'), legend=alt.Legend(title="Porportion not Represented")),
        tooltip = [alt.Tooltip('ST',title='State'),
                   alt.Tooltip('porp_text',title='Porportion Lost'),
                   alt.Tooltip('dist_text',title='District Deficit')
                  ]
    ).add_selection(selection).properties(title='Severity of Partisan Misrepresentation')
    #set up for wave sensitive results
    conv = call_wave(wave)
    dist = dd[dd["year"] == year]
    dist = dist.merge(conv, how="left")
    #create the amounts chart for the Majority Bar
    amounts = pd.DataFrame(dist.prob.value_counts()).reset_index()
    amounts.columns = ["prob_GOP","num_dist"]
    amounts["quantity"] = amounts["prob_GOP"] * amounts["num_dist"]
    expec_GOP = amounts["quantity"].sum()
    expec_dem = 435 - expec_GOP
    d = {'Expected Seats' : [expec_dem, expec_GOP], 'Party' : ["Dem","GOP"]}
    expec = pd.DataFrame(data = d, index=None)
    #output an associated phrase
    party_control = str(np.where(expec_GOP > expec_dem, "Republicans", "Democrats"))
    num = np.where(expec_GOP > expec_dem, expec_GOP, expec_dem)
    num_seats = str(num.round())
    phrase = party_control + " are expected to control the house with a majority of " + num_seats.rstrip(".0")  + " seats"
    #output majority control chart
    cd_scale = [0,50,100,150,175,200,218,235,260,285,335,385,435]
    maj = alt.Chart(expec).mark_bar().encode(
        alt.X('sum(Expected Seats)',axis=alt.Axis(values=cd_scale),title="Expected Partisan Control"),
        alt.Color('Party', scale=alt.Scale(range=["blue","red"]))
    ).properties(title=phrase)
    #format data for median seat chart
    dist = dist.sort_values("metric",ascending=False).reset_index()
    dist["X"] = hc["X"]
    dist["Y"] = hc["Y"]
    hg = alt.Chart(dist).mark_point().encode(
        alt.X('X',axis=None),
        alt.Y('Y',axis=None),
        #alt.Text('ST'),
        alt.Color('prob', scale=alt.Scale(range=["darkblue","blue","grey","red","darkred"]), legend=alt.Legend(title="Probability of GOP Representation")),
        tooltip = [alt.Tooltip('ST#',title='District'),
                   alt.Tooltip('PVI',title='PVI'),
                   alt.Tooltip('prob',title='Prob GOP'),
                  ]
    ).configure_point(size=15,shape=hexagon,filled=True).configure_view(strokeWidth=0)
    if chart == "Cartogram":
        return cart
    elif chart == "Bubble":
        return bubble
    elif chart == "Median":
        return hg
    else:
        return maj

In [8]:
get_viz(2022,"Democratic","Cartogram")

<ipython-input-7-4d04154a63e7>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["ST"] = df["ST"]
<ipython-input-5-56a58d351697>:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  index["prob"] = col


alt.LayerChart(...)

In [9]:
cv[5]

KeyError: 5